
## Legal Research Assistant Bot about the US Constitution

Function operations do the following:

* Read the US_Constitution.txt file inside the some_data folder
* Split this into chunks (you choose the size)
* Write this to a ChromaDB Vector Store
* Use Context Compression to return the relevant portion of the document to the question

In [28]:
import os

import warnings
warnings.filterwarnings("ignore")

from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 


In [29]:
def us_constitution_helper(question):
    '''
    Takes in a question about the US Constitution and returns the most relevant
    part of the constitution. Notice it may not directly answer the actual question!
    
    '''
    api_key = os.getenv('OPENAI_API_KEY')

    # PART ONE:
    # LOAD "some_data/US_Constitution in a Document object
    loader = TextLoader("some_data/US_Constitution.txt")
    documents = loader.load()
    
    # PART TWO
    # Split the document into chunks (you choose how and what size)
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=500)
    docs = text_splitter.split_documents(documents) 
    
    # PART THREE
    # EMBED THE Documents (now in chunks) to a persisted ChromaDB
    embedding_function = OpenAIEmbeddings(open_api_key = api_key)
    db = Chroma.from_documents(docs, embedding_function,persist_directory='./us_constitution_db')
    db.persist()

    # PART FOUR
    # Use ChatOpenAI and ContextualCompressionRetriever to return the most
    # relevant part of the documents.
    llm = ChatOpenAI(temperature=0)
    compressor = LLMChainExtractor.from_llm(llm)
    db_connection = Chroma(persist_directory='./us_constitution_db',embedding_function=embedding_function)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db_connection.as_retriever())
    
    compressed_docs = compression_retriever.get_relevant_documents("What is the 13th Amendment?")


    return compressed_docs[0].page_content

In [30]:
print(us_constitution_helper("What is the 13th Amendment?"))

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-bgzQ14oKqPhXJrSz9hJVea4P on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-gpt-3.5-turbo in organization org-bgzQ14oKqPhXJrSz9hJVea4P on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit ht

13th Amendment
Section 1
Neither slavery nor involuntary servitude, except as a punishment for crime whereof the party shall have been duly convicted, shall exist within the United States, or any place subject to their jurisdiction.
